## 11. Transformix: Spatial Jacobian and Deformation Field calculations

With the transformix algorithm the spatial jacobian, the determinant of the spatial jacobian and the deformation field of the transformation can be calculated.
Especially the determinant of the spatial Jacobian, which identifies the amount of local
compression or expansion and can be quite useful, for example in lung ventilation studies.

The deformation field is a vector image where each voxel contains the displacement vector in physical coordinates.

### Elastix

In [32]:
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk

In [33]:
# Import Images
fixed_image = itk.imread('data/CT_2D_head_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_2D_head_moving.mha', itk.F)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(parameter_map_rigid)

Registration with the registration function.
The output directory has to be specified, 
otherwise elastix will not save the transformparameter file as .txt file

In [36]:
# Call registration function and specify output directory
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    output_directory='exampleoutput/')

### Transformix

In [35]:
# Import Image to transform, transformix is transforming from moving -> fixed;
# for this example the exact same moving image is used, this however is normally not 
# very usefull since the elastix algorithm already transformed this image.
moving_image_transformix = itk.imread('data/CT_2D_head_moving.mha', itk.F)

Transformation can either be done in one line with the transformix function...

In [27]:
transformed_image = itk.transformix_filter(
    moving_image=moving_image_transformix,
    transform_parameter_object=result_transform_parameters,
    compute_deformation_field=True,
    compute_spatial_jacobian=True,
    compute_determinant_of_spatial_jacobian=True,
    output_directory='exampleoutput/')

.. or by initiating an transformix image filter object.

In [28]:
# Load Transformix Object
transformix_object = itk.TransformixFilter.New()
transformix_object.SetMovingImage(moving_image_transformix)
transformix_object.SetTransformParameterObject(result_transform_parameters)

# Set advanced options
transformix_object.SetComputeSpatialJacobian(True)
transformix_object.SetComputeDeterminantOfSpatialJacobian(True)
transformix_object.SetComputeDeformationField(True)

# Set output directory for spatial jacobian and its determinant,
# default directory is current directory.
transformix_object.SetOutputDirectory('exampleoutput/')

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
result_image_transformix = transformix_object.GetOutput()
deformation_field = transformix_object.GetOutputDeformationField()

The deformation field can currently only be computed with the object oriented method of transformix, this will change is future updates to ITKElastix

## Deformation Field Inversion

The DisplacementMagnitudePenalty is a cost function that penalises ||Tµ(x) − x||2. You can use this
to invert transforms, by setting the transform to be inverted as an initial transform (using -t0), setting
(HowToCombineTransforms "Compose"), and running elastix with this metric, using the original fixed
image set both as fixed (-f) and moving (-m) image. After that you can manually set the initial transform
in the last parameter file to "NoInitialTransform", and voila, you have the inverse transform! Strictly
speaking, you should then also change the Size/Spacing/Origin/Index/Direction settings to match that of
the moving image.
Note that inverting a transformation becomes conceptually very similar to performing an image registration
in this way. Consequently, the same choices are relevant: optimisation algorithm, multiresolution etc...
With Transformix the inverted transform can then be used to calculate the inversion of the deformation field, just like Transformix normally calculates the deformation field from a transform.

In [43]:
# Import Default Parameter Map and adjust parameters
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('rigid')
parameter_map_rigid['HowToCombineTransforms'] = ['Compose']
parameter_object.AddParameterMap(parameter_map_rigid)

# Call registration function with transform parameters of normal elastix run as initial transform
# on fixed image to fixed image registration.
inverse_image, inverse_transform_parameters = itk.elastix_registration_method(
    fixed_image, fixed_image,
    parameter_object=parameter_object,
    initial_transform_parameter_file_name='exampleoutput/TransformParameters.0.txt')

# Adjust inverse transform parameters object
inverse_transform_parameters.SetParameter(
    0, "InitialTransformParametersFileName", "NoInitialTransform")

In [44]:
# Load Transformix Object with inverse transform parameters
transformix_object = itk.TransformixFilter.New()

# Setting the original fixed image for inverse transformation should give the moving image as result
# but since we're focussed on obtaining the inverse deformation field, setting this image is trivial.
transformix_object.SetMovingImage(fixed_image)
transformix_object.SetTransformParameterObject(inverse_transform_parameters)

# Set Deformation Field boolean
transformix_object.SetComputeDeformationField(True)

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Get the inverse of the deformation field
inverse_deformation_field = transformix_object.GetOutputDeformationField()